In [138]:
!pip install emoji

In [0]:
import emoji

In [140]:
emoji.EMOJI_UNICODE


{':1st_place_medal:': '🥇',
 ':2nd_place_medal:': '🥈',
 ':3rd_place_medal:': '🥉',
 ':AB_button_(blood_type):': '🆎',
 ':ATM_sign:': '🏧',
 ':A_button_(blood_type):': '🅰',
 ':Afghanistan:': '🇦🇫',
 ':Albania:': '🇦🇱',
 ':Algeria:': '🇩🇿',
 ':American_Samoa:': '🇦🇸',
 ':Andorra:': '🇦🇩',
 ':Angola:': '🇦🇴',
 ':Anguilla:': '🇦🇮',
 ':Antarctica:': '🇦🇶',
 ':Antigua_&_Barbuda:': '🇦🇬',
 ':Aquarius:': '♒',
 ':Argentina:': '🇦🇷',
 ':Aries:': '♈',
 ':Armenia:': '🇦🇲',
 ':Aruba:': '🇦🇼',
 ':Ascension_Island:': '🇦🇨',
 ':Australia:': '🇦🇺',
 ':Austria:': '🇦🇹',
 ':Azerbaijan:': '🇦🇿',
 ':BACK_arrow:': '🔙',
 ':B_button_(blood_type):': '🅱',
 ':Bahamas:': '🇧🇸',
 ':Bahrain:': '🇧🇭',
 ':Bangladesh:': '🇧🇩',
 ':Barbados:': '🇧🇧',
 ':Belarus:': '🇧🇾',
 ':Belgium:': '🇧🇪',
 ':Belize:': '🇧🇿',
 ':Benin:': '🇧🇯',
 ':Bermuda:': '🇧🇲',
 ':Bhutan:': '🇧🇹',
 ':Bolivia:': '🇧🇴',
 ':Bosnia_&_Herzegovina:': '🇧🇦',
 ':Botswana:': '🇧🇼',
 ':Bouvet_Island:': '🇧🇻',
 ':Brazil:': '🇧🇷',
 ':British_Indian_Ocean_Territory:': '🇮🇴',
 ':British_Virgin_Is

In [0]:
emoji_dictionary={
    "0":"\u2764\uFE0F",
    "1":":baseball:",
    "2":":grinning_face_with_big_eyes:",
    "3":":disappointed_face:",
    "4":":fork_and_knife:",
    
}


In [142]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😃
😞
🍴


In [0]:
import pandas as pd
import numpy as np

In [0]:
train=pd.read_csv("train_emoji.csv",header=None)

In [0]:
test=pd.read_csv("test_emoji.csv",header=None)

In [146]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [147]:
X_train=train[0] #training x
Y_train=train[1] #training y
x_train=test[0] #test x
y_test=test[1] #test y
from keras.utils import to_categorical
Y_train=to_categorical(Y_train,num_classes=5)
y_test=to_categorical(y_test,num_classes=5)    #here i have converted the label into one hot vector  which holds 5 emoji classes
print(Y_train.shape)


(132, 5)


In [149]:
for i in range(5):
    print(x_train[i],emoji.emojize(emoji_dictionary[str(y_test[i])]))


KeyError: ignored

In [0]:
f=open("glove.6B.50d.txt",encoding='utf-8')

In [0]:
embeddings_index={}
for line in f:
    values=line.split()
    word=values[0]
    coeff=np.asarray(values[1:],dtype='float')
    embeddings_index[word]=coeff
f.close()

In [152]:
ndim=embeddings_index["cow"].shape[0]
print(ndim)

50


In [0]:
def embedding_output(X):
    #this is the outpputof the embedding layer jaha har sentence vec ki shape me banke rnn cell me jayega in form of batch
    maxlen=10
    embedding_output=np.zeros((X.shape[0],maxlen,ndim))
    for ix in range(X.shape[0]):
        X[ix] = str(X[ix]).split()
        
        
        for ij in range(len(X[ix])):
            try:
                embedding_output[ix][ij]=embeddings_index[X[ix][ij].lower()]
            except:
                embedding_output[ix][ij]=np.zeros((50,))
    return embedding_output

In [154]:
embedding_matrix_train=embedding_output(X_train)
embedding_matrix_test=embedding_output(x_train)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [155]:
print(embedding_matrix_train.shape)   #ye vo cube format h jo embedding layer output me degi poore batch k har sentence ko vector me show kr k

print(embedding_matrix_test.shape)

(132, 10, 50)
(56, 10, 50)


In [0]:
from keras.models import Sequential
from keras.layers import *

In [186]:
model=Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True)) #64 shape output hum next cell ko bhejenge it can be any numebr aur ye jo (10,50 diya h ye har sentence ka dimension h 10 words *50 vector represesntation of each word)
model.add(Dropout(0.5))
model.add(LSTM(64,return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_9 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dropout_10 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_9 (Activation)    (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
__________________________________________________

In [0]:
from keras.utils import to_categorical
#from keras.utils import ModelCheckpoint
#checkpoint=ModelCheckpoint("best_model.h5",monitor='val_loss',verbose=True,save_best_only=True)
from keras.callbacks import EarlyStopping
earlystop=EarlyStopping(monitor='val_acc',patience=10)  #if my model does not increase val_accuracy even after 10 epochs,then i will stop training further

In [188]:
hist=model.fit(embedding_matrix_train,Y_train,epochs=100,batch_size=64,shuffle=True,validation_split=0.2)

Train on 105 samples, validate on 27 samples
Epoch 1/100
105/105 [==============================] - 1s 7ms/step - loss: 1.5871 - accuracy: 0.2762 - val_loss: 1.5914 - val_accuracy: 0.2222
Epoch 2/100
105/105 [==============================] - 0s 505us/step - loss: 1.5398 - accuracy: 0.3619 - val_loss: 1.5998 - val_accuracy: 0.2222
Epoch 3/100
105/105 [==============================] - 0s 535us/step - loss: 1.5192 - accuracy: 0.3714 - val_loss: 1.6137 - val_accuracy: 0.1852
Epoch 4/100
105/105 [==============================] - 0s 478us/step - loss: 1.4934 - accuracy: 0.3524 - val_loss: 1.6324 - val_accuracy: 0.2222
Epoch 5/100
105/105 [==============================] - 0s 483us/step - loss: 1.4733 - accuracy: 0.3810 - val_loss: 1.6463 - val_accuracy: 0.2593
Epoch 6/100
105/105 [==============================] - 0s 513us/step - loss: 1.4614 - accuracy: 0.4000 - val_loss: 1.6507 - val_accuracy: 0.2593
Epoch 7/100
105/105 [==============================] - 0s 554us/step - loss: 1.4238 - a

In [0]:
predictions=model.predict_classes(embedding_matrix_test)

In [190]:
print(predictions)

[4 3 3 2 2 2 3 2 4 2 1 2 0 3 1 3 2 2 3 2 3 0 4 0 3 3 2 0 1 2 0 1 3 2 0 2 2
 3 4 2 1 0 0 1 2 2 2 2 3 3 3 0 3 2 2 0]


In [191]:
model.evaluate(embedding_matrix_test,y_test)

56/56 [==============================] - 0s 201us/step


[2.09718542439597, 0.5892857313156128]

In [192]:
for i in range(5):
  print(x_train[i],emoji.emojize(emoji_dictionary[str(predictions[i])]))


['I', 'want', 'to', 'eat'] 🍴
['he', 'did', 'not', 'answer'] 😞
['he', 'got', 'a', 'raise'] 😞
['she', 'got', 'me', 'a', 'present'] 😃
['ha', 'ha', 'ha', 'it', 'was', 'so', 'funny'] 😃
